In [1]:
from __future__ import division
import tensorflow as tf
import numpy as np
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
import scipy.io
import gc
import matplotlib.pyplot as plt
from my_utils import *

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
sur_mat = sorted (os.listdir('surreal_data/masks'))
sur_jpg = sorted( os.listdir('surreal_data/images'))
mat_path = []
for i in range(len(sur_mat)):
    temp_path = os.path.join('surreal_data/masks',sur_mat[i])
    mat_path.append(temp_path)
jpg_path = []
for i in range(len(sur_jpg)):
    temp_path = os.path.join('surreal_data/images',sur_jpg[i])
    jpg_path.append(temp_path)
data_table = pd.DataFrame(jpg_path,columns=['jpg'])
mat_frame = pd.DataFrame(mat_path,columns=['mat'])
data_table = pd.concat([data_table,mat_frame],axis=1)
data_table.head()

,jpg,mat
0,surreal_data/images\0.jpg,surreal_data/masks\0.mat
1,surreal_data/images\1.jpg,surreal_data/masks\1.mat
2,surreal_data/images\10.jpg,surreal_data/masks\10.mat
3,surreal_data/images\100.jpg,surreal_data/masks\100.mat
4,surreal_data/images\1000.jpg,surreal_data/masks\1000.mat


In [3]:
data_table.to_csv('surreal_data_table.csv')

In [179]:
# load images and masks
# (240,320,3)
Data = np.zeros([30,256,256,4])
for i in range(10):
    test_img = image.load_img(data_table['jpg'][i])
    test_img = image.img_to_array(test_img)
    test_mask = scipy.io.loadmat(data_table['mat'][i])['mask']
    crop1, crop2, crop3 = random_put_into_256map(test_img,test_mask)
    Data[i*3] = crop1
    Data[i*3+1] = crop2
    Data[i*3+2] = crop3

In [180]:
# split the data into train and validation
rnd = np.arange(len(Data))
train_idx = rnd < len(Data)*0.8
valid_idx = rnd >= len(Data)*0.8
train_data = Data[train_idx]
val_data = Data[valid_idx]
# save some memory
del Data
gc.collect()

46953

##---test---

In [194]:
void_map = from_head_scale(train_data[20])

In [195]:
plt.figure()
plt.imshow(void_map[:,:,0:3]/255)
plt.show()
plt.figure()
plt.imshow(void_map[:,:,3])
plt.show()

In [ ]:
aimg,amask = augmentor_with_keras(train_data[20][:,:,0:3],train_data[20][:,:,3])
plt.figure()
plt.imshow(aimg/285)
plt.show()
plt.figure()
plt.imshow(amask)
plt.show()

##---test end---


In [12]:
def process_config(conf_file):
	"""
	"""
	params = {}
	config = configparser.ConfigParser()
	config.read(conf_file)
	for section in config.sections():
		if section == 'DataSetHG':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Network':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Train':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Validation':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Saver':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
	return params
print('--Parsing Config File')
params = process_config('config.cfg')

--Parsing Config File


In [13]:
model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'], 
                       nModules=params['nmodules'], 
                       nLow=params['nlow'], outputDim=params['num_joints'], 
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'], 
                       lear_rate=params['learning_rate'], 
                       decay=params['learning_rate_decay'], 
                       decay_step=params['decay_step'], 
                       name=params['name'], 
                       logdir_train=params['log_dir_train'], 
                       logdir_test=params['log_dir_test'], 
                       tiny= params['tiny'], w_loss=params['weighted_loss'] , 
                       joints= params['joint_list'],modif=False)
model.generate_model()

CREATE MODEL:
---Inputs : Done (0 sec.)


---Graph : Done (43 sec.)
---Loss : Done (0 sec.)


In [14]:
# output: (None,8,64,64,15)
output = model.get_output()
loss = model.loss
out = output[0,7]
pred_seg = tf.argmax(out,axis=2)
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)
saver = tf.train.Saver()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [15]:
# train
for i in range(30):
    x_b,y_b = get_next_batch(train_data)
    x,y = get_augmentor_batch(x_b,y_b,3)
    sess.run(train_step,feed_dict={model.img:x, model.gtMaps:y})

    x_for_eval = np.expand_dims(x_b,axis=0)
    pred = sess.run(pred_seg,feed_dict={model.img:x_for_eval})
    acc = cal_acc(pred,y_b)
    iou = cal_iou(pred,y_b)
    print('acc:%f',acc,'  ','iou:%f',iou)
    print('training loss:',sess.run(model.loss, feed_dict={model.img:x, model.gtMaps:y}))

acc:%f 0.010009765625    iou:%f 0.0


training loss: 0.7933299


acc:%f 0.006591796875    iou:%f 0.0


training loss: 0.7935052


acc:%f 0.005126953125    iou:%f 0.0


...

acc:%f 0.007080078125    iou:%f 0.0


training loss: 0.791947


acc:%f 0.008056640625    iou:%f 0.0


training loss: 0.79311144


KeyboardInterrupt: 